Exploring some intrinsic dimension (ID) and local intrinsic dimension (LID) estimators for high-dimensional data.

### References
1. Ma, Xingjun, et al. "Characterizing adversarial subspaces using local intrinsic dimensionality." arXiv preprint arXiv:1801.02613 (2018).
1. Amsaleg, Laurent, et al. "Estimating local intrinsic dimensionality." Proceedings of the 21th ACM SIGKDD International Conference on Knowledge Discovery and Data Mining. ACM, 2015.
1. Ansuini, Alessio, et al. "Intrinsic dimension of data representations in deep neural networks." arXiv preprint arXiv:1905.12784 (2019).
1. Carter, Kevin M., Raviv Raich, and Alfred O. Hero III. "On local intrinsic dimension estimation and its applications." IEEE Transactions on Signal Processing 58.2 (2009): 650-663.
1. Levina, Elizaveta, and Peter J. Bickel. "Maximum likelihood estimation of intrinsic dimension." Advances in neural information processing systems. 2005.
1. Facco, Elena, et al. "Estimating the intrinsic dimension of datasets by a minimal neighborhood information." Scientific reports 7.1 (2017): 12140.

### LID estimation method from [1] and [2]

In [1]:
import numpy as np
import os
from pynndescent import NNDescent
from sklearn.neighbors import NearestNeighbors
from multiprocessing import cpu_count
from sklearn.model_selection import StratifiedShuffleSplit
from metrics_custom import remove_self_neighbors
from generate_data import MFA_model
from lid_estimators import (
    lid_mle_amsaleg, 
    id_two_nearest_neighbors, 
    estimate_intrinsic_dimension
)
from dimension_reduction_methods import pca_wrapper

In [2]:
# Suppress annoying numba warning
import warnings
from numba import NumbaPendingDeprecationWarning
warnings.filterwarnings('ignore', '', NumbaPendingDeprecationWarning)

In [3]:
# Define some constants
num_proc = max(cpu_count() - 2, 1)
seed_rng = np.random.randint(1, high=10000)
K = 20
n_neighbors = max(K + 2, 20)
rho = 0.5
metric_primary = 'euclidean'

In [4]:
# Generate data according to a mixture of factor analysis (MFA) model
np.random.seed(seed_rng)

# number of mixture components
n_components = 10
# dimension of the observed space
dim = 500

# dimension of the latent space. This determines the local intrinsic dimension
# dim_latent = 10
# model = MFA_model(n_components, dim, dim_latent=dim_latent, seed_rng=seed_rng)

# Can specify a range for the latent dimension instead of a single value
dim_latent_range = (10, 20)
model = MFA_model(n_components, dim, dim_latent_range=dim_latent_range, seed_rng=seed_rng)

# Generate data from the model
N = 1000
N_test = 100
data, labels = model.generate_data(N)
data_test, labels_test = model.generate_data(N_test)

In [5]:
# Construct an approximate nearest neighbor (ANN) index to query nearest neighbors
params = {
    'metric': metric_primary, 
    'n_neighbors': n_neighbors,
    'rho': rho,
    'random_state': seed_rng,
    'n_jobs': num_proc, 
    'verbose': True
}
index = NNDescent(data, **params)

Tue Dec 24 10:18:46 2019 Building RP forest with 7 trees
Tue Dec 24 10:18:47 2019 parallel NN descent for 10 iterations
	 0  /  10
	 1  /  10
	 2  /  10
	 3  /  10


In [6]:
# Query the K nearest neighbors of each point. 
# Since each point will be selected as its own nearest neighbor, we query for `K+1` neighbors and ignore the self neighbors
nn_indices_, nn_distances_ = index.query(data, k=(K + 1))

# Remove each point from it's own neighborhood set
nn_indices, nn_distances = remove_self_neighbors(nn_indices_, nn_distances_)

In [7]:
# Calculate the local intrinsic dimension in the neighborhood of each point
lid = lid_mle_amsaleg(nn_distances)
print("Mean LID = {:.4f}".format(np.mean(lid)))

Mean LID = 10.5029


In [8]:
p = [0, 2.5, 25, 50, 75, 97.5, 100]
out = np.percentile(lid, p)
print("Percentiles of the LID distribution:")
for a, b in zip(p, out):
    print("{:.1f}\t{:.4f}".format(a, b))

Percentiles of the LID distribution:
0.0	1.8432
2.5	4.3697
25.0	8.4421
50.0	10.2791
75.0	12.1687
97.5	17.8415
100.0	24.5904


### Intrinsic dimension estimation using the Two-NN method [6, 3]

In [9]:
id = id_two_nearest_neighbors(nn_distances)
print("Intrinsic dimension estimated using the two-NN method = {:.4f}".format(id))

Intrinsic dimension estimated using the two-NN method = 14.7769


Observe how the intrinsic dimension and local intrinsic dimension estimates are below `20` despite the (ambient) dimension of the data being `500`. This is consistent with the underlying MFA model whose latent dimension from each component is chosen uniformly from the range `[10, 20]`.

In [10]:
# Testing the wrapper function which essentially does the above steps under the hood
id1 = estimate_intrinsic_dimension(data, method='two_nn', n_neighbors=K, n_jobs=num_proc, seed_rng=seed_rng)
id2 = estimate_intrinsic_dimension(data, method='lid_mle', n_neighbors=K, n_jobs=num_proc, seed_rng=seed_rng)

print("Intrinsic dimension estimates from both the methods: {:.4f}, {:.4f}".format(id1, id2))

Intrinsic dimension estimates from both the methods: 14.7769, 10.2791


### MNIST data

In [11]:
from sklearn.datasets import fetch_openml

data_all, labels_all = fetch_openml('mnist_784', version=1, return_X_y=True)

# Scale the feature values to the range [0, 1]
data_all = data_all / 255.

# Create a random, stratified train/test split
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=seed_rng)
data = labels = None
data_test = labels_test = None
for index_tr, index_te in sss.split(data_all, labels_all):
    data = data_all[index_tr, :]
    labels = labels_all[index_tr]
    data_test = data_all[index_te, :]
    labels_test = labels_all[index_te]

print("Number of train samples = {:d}. Number of dimensions = {:d}".format(*data.shape))

Number of train samples = 35000. Number of dimensions = 784


In [12]:
# Applying PCA as a preprocessing step to retain dimensions that account for 99% of the variance
data_proj, mean_data, transform_pca = pca_wrapper(data, cutoff=0.99, seed_rng=seed_rng)
print("Dimension of the PCA transformed data = {:d}".format(data_proj.shape[1]))

id = estimate_intrinsic_dimension(data_proj, method='two_nn', n_neighbors=2, n_jobs=num_proc, 
                                  seed_rng=seed_rng)
print("Intrinsic dimension estimate using the two-NN method = {:.4f}".format(id))

id = estimate_intrinsic_dimension(data_proj, method='lid_mle', neighborhood_constant=0.4, n_jobs=num_proc, 
                                  seed_rng=seed_rng)
print("Intrinsic dimension estimate using the maximum likelihood method = {:.4f}".format(id))

INFO:dimension_reduction_methods:Number of nonzero singular values in the data matrix = 784
INFO:dimension_reduction_methods:Number of principal components accounting for 99.0 percent of the data variance = 330
INFO:dimension_reduction_methods:Dimension of the PCA transformed data = 330


Dimension of the PCA transformed data = 330
Intrinsic dimension estimate using the two-NN method = 14.2012
Intrinsic dimension estimate using the maximum likelihood method = 12.5246


### CIFAR-10 data

In [13]:
import sys
data_path_local = '/Users/jayaram/Documents/research/data/cifar-10-batches-py'
if data_path_local not in sys.path:
    sys.path.append(data_path_local)
    
from load_cifar10_data import load_cifar_10_data
data, labels, _, data_test, labels_test, _, _ = load_cifar_10_data(data_path_local, vectorize=True)

# Scale the feature values to the range [0, 1]
data = data / 255.
data_test = data_test / 255.
print("Number of train samples = {:d}. Number of dimensions = {:d}".format(*data.shape))

Number of train samples = 50000. Number of dimensions = 3072


In [14]:
# Applying PCA as a preprocessing step to retain dimensions that account for 99% of the variance
data_proj, mean_data, transform_pca = pca_wrapper(data, cutoff=0.99, seed_rng=seed_rng)
print("Dimension of the PCA transformed data = {:d}".format(data_proj.shape[1]))

id = estimate_intrinsic_dimension(data_proj, method='two_nn', n_neighbors=2, n_jobs=num_proc, 
                                  seed_rng=seed_rng)
print("Intrinsic dimension estimate using the two-NN method = {:.4f}".format(id))

id = estimate_intrinsic_dimension(data_proj, method='lid_mle', neighborhood_constant=0.4, n_jobs=num_proc, 
                                  seed_rng=seed_rng)
print("Intrinsic dimension estimate using the maximum likelihood method = {:.4f}".format(id))

INFO:dimension_reduction_methods:Number of nonzero singular values in the data matrix = 3072
INFO:dimension_reduction_methods:Number of principal components accounting for 99.0 percent of the data variance = 658
INFO:dimension_reduction_methods:Dimension of the PCA transformed data = 658


Dimension of the PCA transformed data = 658
Intrinsic dimension estimate using the two-NN method = 7.9672
Intrinsic dimension estimate using the maximum likelihood method = 26.6976


### SVHN data

In [15]:
# Load the data from a local copy
data_path_local = '/Users/jayaram/Documents/research/data/SVHN'
if data_path_local not in sys.path:
    sys.path.append(data_path_local)
    
from preprocess_svhn import load_images, normalize_images

imgs_train, imgs_test = load_images(data_path_local)
data, labels = normalize_images(imgs_train, vectorize=True)
data_test, labels_test = normalize_images(imgs_test, vectorize=True)

print("Number of train samples = {:d}. Number of dimensions = {:d}".format(*data.shape))

Number of train samples = 73257. Number of dimensions = 3072


In [16]:
# Applying PCA as a preprocessing step to retain dimensions that account for 99.5% of the variance
data_proj, mean_data, transform_pca = pca_wrapper(data, cutoff=0.995, seed_rng=seed_rng)
print("Dimension of the PCA transformed data = {:d}".format(data_proj.shape[1]))

id = estimate_intrinsic_dimension(data_proj, method='two_nn', n_neighbors=2, n_jobs=num_proc, 
                                  seed_rng=seed_rng)
print("Intrinsic dimension estimate using the two-NN method = {:.4f}".format(id))

id = estimate_intrinsic_dimension(data_proj, method='lid_mle', neighborhood_constant=0.4, n_jobs=num_proc, 
                                  seed_rng=seed_rng)
print("Intrinsic dimension estimate using the maximum likelihood method = {:.4f}".format(id))

INFO:dimension_reduction_methods:Number of nonzero singular values in the data matrix = 3072
INFO:dimension_reduction_methods:Number of principal components accounting for 99.5 percent of the data variance = 266
INFO:dimension_reduction_methods:Dimension of the PCA transformed data = 266


Dimension of the PCA transformed data = 266


/Users/jayaram/Documents/research/code/KNN-anomaly-detection/lid_estimators.py:48: RuntimeWarning: overflow encountered in true_divide
  nn_ratio = knn_distances[:, 1] / np.clip(knn_distances[:, 0], sys.float_info.min, None)
/anaconda3/envs/knn_expts/lib/python3.7/site-packages/numpy/lib/function_base.py:2445: RuntimeWarning: invalid value encountered in subtract
  X -= avg[:, None]


Intrinsic dimension estimate using the two-NN method = nan
Intrinsic dimension estimate using the maximum likelihood method = 18.7266
